In [3]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

In [4]:
#torch.cuda.is_available   #检查cuda环境是否存在
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
#超参数
input_size = 784      #28*28
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [20]:
#加载数据 MNIST
train_dataset = torchvision.datasets.MNIST(root = '../../data',
                                                            train = True,
                                                            transform = transforms.ToTensor(),
                                                            download = False)
test_dataset = torchvision.datasets.MNIST(root = '../../data',
                                         train = False,
                                         transform = transforms.ToTensor())



In [21]:
#数据下载器，加载数据
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = batch_size,
                                         shuffle = False)

In [7]:
#包含一个隐藏层的全连接网络
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [12]:
model = NeuralNet(input_size, hidden_size, num_classes)

In [13]:
#实例化一个损失函数对象
criterion = nn.CrossEntropyLoss()

#实例化一个优化器对象
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [14]:
#训练模型
total_step = len(train_loader)   #计算总共有多少个batch
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        #将张量移动到设置好的设备中去（cpu或gpu）
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        #前向
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        #后向，优化
        optimizer.zero_grad()  #梯度清零
        loss.backward()
        optimizer.step()
        
        if (i + 1)%100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch +1, num_epochs, i+1, total_step, loss.item()))
            
        


Epoch [1/5], Step [100/600], Loss: 0.4087
Epoch [1/5], Step [200/600], Loss: 0.2693
Epoch [1/5], Step [300/600], Loss: 0.1944
Epoch [1/5], Step [400/600], Loss: 0.1634
Epoch [1/5], Step [500/600], Loss: 0.1741
Epoch [1/5], Step [600/600], Loss: 0.2396
Epoch [2/5], Step [100/600], Loss: 0.1160
Epoch [2/5], Step [200/600], Loss: 0.1064
Epoch [2/5], Step [300/600], Loss: 0.0966
Epoch [2/5], Step [400/600], Loss: 0.2069
Epoch [2/5], Step [500/600], Loss: 0.0622
Epoch [2/5], Step [600/600], Loss: 0.0832
Epoch [3/5], Step [100/600], Loss: 0.1221
Epoch [3/5], Step [200/600], Loss: 0.0393
Epoch [3/5], Step [300/600], Loss: 0.0375
Epoch [3/5], Step [400/600], Loss: 0.0627
Epoch [3/5], Step [500/600], Loss: 0.0725
Epoch [3/5], Step [600/600], Loss: 0.1683
Epoch [4/5], Step [100/600], Loss: 0.0671
Epoch [4/5], Step [200/600], Loss: 0.0209
Epoch [4/5], Step [300/600], Loss: 0.0954
Epoch [4/5], Step [400/600], Loss: 0.0452
Epoch [4/5], Step [500/600], Loss: 0.0365
Epoch [4/5], Step [600/600], Loss:

In [22]:
#测试模型，在测试阶段，不需要计算和更新梯度（节省内存消耗）
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print ('Accuracy of the network on the 10000 test images:{}%'.format(100*correct / total))
    
    

Accuracy of the network on the 10000 test images:97.85%


In [17]:
#保存模型
torch.save(model.state_dict(), 'model.ckpt')